In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,Dataset
import os 
import matplotlib.pyplot as plt

In [26]:
data_path='D:\AI_Diploma\AI_diploma\Computer_Vision\Projects\Blood Cells Image Dataset'


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset

class ImageData(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.image_paths = []
        self.labels = []
        self.class_name = {}
        self.transform = transform

        for label, class_dir in enumerate(os.listdir(data_path)):
            class_path = os.path.join(data_path, class_dir)
            if not os.path.isdir(class_path):
                continue 

            self.class_name[label] = class_dir

            for image in os.listdir(class_path):
                image_path = os.path.join(class_path, image)
                self.image_paths.append(image_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [35]:
show=ImageData(data_path)

In [43]:
image,label=show[1000]
label

1

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# تحويلات الصور
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),         
])

dataset = ImageData(data_path, transform=transform)

#  Train / Test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)   #
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> 64x64
        x = self.pool(F.relu(self.conv2(x)))  # -> 32x32
        x = self.pool(F.relu(self.conv3(x)))  # -> 16x16
        x = x.view(x.size(0), -1)             # Flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x



In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(dataset.class_name)
model = CNN_Model(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 1.1380
Epoch [2/10], Loss: 0.5124
Epoch [3/10], Loss: 0.4115
Epoch [4/10], Loss: 0.3349
Epoch [5/10], Loss: 0.2892
Epoch [6/10], Loss: 0.2399
Epoch [7/10], Loss: 0.2082
Epoch [8/10], Loss: 0.1703
Epoch [9/10], Loss: 0.1377
Epoch [10/10], Loss: 0.1157


In [30]:
torch.save(model.state_dict(), "cnn_model.pth")

In [ ]:
correct = 0
total = 0

model.eval()  

with torch.no_grad():  
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)          
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ Test Accuracy: {accuracy:.2f}%")

✅ Test Accuracy: 94.92%
